In [2]:
import os
os.chdir('../')
%pwd

'c:\\Users\\LONG\\Computer Science\\Artificial Intelligence\\Projects\\End-To-End-Chicken-Disease-Classification'

In [3]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [4]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [ ]:
class ConfigurationManager:
    def __init__(
        self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_prepare_callbacks_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories(
            [Path(model_ckpt_dir), Path(config.tensorboard_root_log_dir)]
        )

        prepare_callbacks_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath),
        )
        return prepare_callbacks_config

In [12]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [13]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config

    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir, f"tb_logs_at_{timestamp}"
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath, save_best_only=True
        )

    def get_tb_ckpt_callbacks(self):
        return [self._create_tb_callbacks, self._create_ckpt_callbacks]

In [14]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callbacks_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()

except Exception as e:
    raise

[2025-05-15 10:33:23,902: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-05-15 10:33:23,917: INFO: common: yaml file: params.yaml loaded successfully]
[2025-05-15 10:33:23,921: INFO: common: created directory at: artifacts]
[2025-05-15 10:33:23,927: INFO: common: created directory at: artifacts\prepare_callbacks\checkpoint_dir]
[2025-05-15 10:33:23,929: INFO: common: created directory at: artifacts\prepare_callbacks\tensorboard_root_log_dir]


In [16]:
import time

timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
print(f"tb_logs_at_{timestamp}")

tb_logs_at_2025-05-15-10-36-26
